In [3]:
!pip install transformers  --quiet

     |████████████████████████████████| 4.7 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 41.2 MB/s 


In [4]:
!pip install datasets --quiet

In [5]:
 !wget https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/ABSA_cricket_train.tsv

--2022-08-20 02:22:52--  https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/ABSA_cricket_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 338729 (331K) [text/plain]
Saving to: ‘ABSA_cricket_train.tsv’

ABSA_cricket_train. 100%[===================>] 330.79K  --.-KB/s    in 0.03s   

2022-08-20 02:22:52 (9.70 MB/s) - ‘ABSA_cricket_train.tsv’ saved [338729/338729]



In [6]:
!wget https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/ABSA_cricket_test.tsv

--2022-08-20 02:22:52--  https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/ABSA_cricket_test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65089 (64K) [text/plain]
Saving to: ‘ABSA_cricket_test.tsv’

ABSA_cricket_test.t 100%[===================>]  63.56K  --.-KB/s    in 0.01s   

2022-08-20 02:22:52 (5.27 MB/s) - ‘ABSA_cricket_test.tsv’ saved [65089/65089]



In [7]:
import pandas as pd

train_df = pd.read_csv("ABSA_cricket_train.tsv", sep = "\t")
test_df = pd.read_csv("ABSA_cricket_test.tsv", sep = "\t")

In [8]:
 train_df.head()

,id,text,class_label
0,2760,আফতাব আপনি ভালো আছেন?,neutral
1,955,মমিনুল হক আর মুশফিক দুজনেই নিজেদের উইকেট জঘন্য...,positive
2,43,মিরপুর এর পিচ এ যে কেও বোলিং করতে পারে,neutral
3,2211,মুদ্রার ঐ পিঠ দেখা হয়ে গেল বাংলাদেশের.দেশের মা...,negative
4,2490,শুভকামনা রাজ্জাক ভাই।,negative


In [9]:
test_df.head()

,id,text,class_label
0,2754,ইমরুল বাদে বাকি তিনজনের আউট মেনেনিতে পারছিনা ?,negative
1,2746,"আফতাব আহমেদ, আপনার সেই ৬টা এখনো ভুলতে পারিনি।",negative
2,21,জরিমানা করা হউক। ৩ মাসের বেতন কর্তন।,negative
3,2143,মামুর পোলারা কোচের বদ নাম করো এবার কোচ কি জিনি...,negative
4,426,হাথুরে বেটা কত্তটা যে খারাপ সিদ্ধান্ত বাংলাদেশ...,negative


In [10]:
total_train_sentences = len(train_df)
total_train_sentences

1942

In [11]:
total_test_sentences = len(test_df)
total_test_sentences

371

In [113]:
train_cat_labels_count = train_df["class_label"].value_counts()
train_cat_labels_count

negative    1415
positive     344
neutral      183
Name: class_label, dtype: int64

In [13]:
test_cat_labels_count = test_df["class_label"].value_counts()
test_cat_labels_count

negative    270
positive     69
neutral      32
Name: class_label, dtype: int64

In [14]:
train_df.loc[1]

id                                                           955
text           মমিনুল হক আর মুশফিক দুজনেই নিজেদের উইকেট জঘন্য...
class_label                                             positive
Name: 1, dtype: object

## Converting *categorical* class label to *numeric*

In [15]:
def cat_to_num(df_column):
  if df_column == "neutral":
    return 0
  elif df_column == "positive":
    return 1
  elif df_column == "negative":
    return 2

In [70]:
class_labels = {
    0 : "Neutral",
    1 : "Positive",
    2 : "Negative"
}

In [16]:
#creating new series as train labels

train_labels = pd.Series(train_df["class_label"].apply(cat_to_num))


In [17]:
 len(train_labels) == len(train_df)

True

In [18]:
num_labels_count = train_labels.value_counts()

In [19]:
num_class_count = train_df["class_label"].value_counts()

In [20]:
num_labels_count

2    1415
1     344
0     183
Name: class_label, dtype: int64

In [21]:
num_class_count

negative    1415
positive     344
neutral      183
Name: class_label, dtype: int64

### adding numeric label to the train_df


In [22]:


train_df["labels"] = train_labels
train_df.head()

,id,text,class_label,labels
0,2760,আফতাব আপনি ভালো আছেন?,neutral,0
1,955,মমিনুল হক আর মুশফিক দুজনেই নিজেদের উইকেট জঘন্য...,positive,1
2,43,মিরপুর এর পিচ এ যে কেও বোলিং করতে পারে,neutral,0
3,2211,মুদ্রার ঐ পিঠ দেখা হয়ে গেল বাংলাদেশের.দেশের মা...,negative,2
4,2490,শুভকামনা রাজ্জাক ভাই।,negative,2


### adding numeric label to test_df

In [23]:
test_labels = pd.Series(test_df["class_label"].apply(cat_to_num))


test_num_labels_count  = test_labels.value_counts()





In [24]:
test_num_labels_count

2    270
1     69
0     32
Name: class_label, dtype: int64

In [25]:
test_cat_labels_count

negative    270
positive     69
neutral      32
Name: class_label, dtype: int64

In [26]:
test_df["labels"] = test_labels
test_df.head()

,id,text,class_label,labels
0,2754,ইমরুল বাদে বাকি তিনজনের আউট মেনেনিতে পারছিনা ?,negative,2
1,2746,"আফতাব আহমেদ, আপনার সেই ৬টা এখনো ভুলতে পারিনি।",negative,2
2,21,জরিমানা করা হউক। ৩ মাসের বেতন কর্তন।,negative,2
3,2143,মামুর পোলারা কোচের বদ নাম করো এবার কোচ কি জিনি...,negative,2
4,426,হাথুরে বেটা কত্তটা যে খারাপ সিদ্ধান্ত বাংলাদেশ...,negative,2


### Creating datafiles for datasets library

In [27]:
!pip install datasets --quiet

In [28]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
train_dataset

Dataset({
    features: ['id', 'text', 'class_label', 'labels'],
    num_rows: 1942
})

In [29]:
test_dataset = Dataset.from_pandas(test_df)
test_dataset

Dataset({
    features: ['id', 'text', 'class_label', 'labels'],
    num_rows: 371
})

In [30]:
checkpoint = "xlm-roberta-base"

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [31]:
tokenizer

PreTrainedTokenizerFast(name_or_path='xlm-roberta-base', vocab_size=250002, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [32]:
def tokenize_function(example):
  return tokenizer(
      example["text"],
      truncation = True
  )

In [33]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched = True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched = True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [34]:
tokenized_train_dataset.features

{'id': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'class_label': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [35]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer = tokenizer, return_tensors = "tf")

In [36]:
sequence = "আমি বেশ টেনশনে আছি"
model_inputs = tokenizer(sequence)
model_inputs

{'input_ids': [0, 21145, 98075, 6, 32115, 2145, 56743, 956, 4979, 38665, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [37]:
tokenizer.decode(model_inputs["input_ids"])

'<s> আমি বেশ টেনশনে আছি</s>'

In [38]:
sequences = ["আমি বেশ টেনশনে আছি","আজ একটি ব্যস্ত দিন যাচ্ছে"]
model_inputs = tokenizer(sequences)
model_inputs

{'input_ids': [[0, 21145, 98075, 6, 32115, 2145, 56743, 956, 4979, 38665, 2], [0, 25508, 14536, 6, 41758, 34619, 29982, 80783, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [39]:
tokenized_train_dataset.features

{'id': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'class_label': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [40]:
tf_train_dataset = tokenized_train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

In [41]:
tf_train_dataset

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [42]:
tf_test_dataset = tokenized_test_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle = False,
    collate_fn=data_collator,
    
    batch_size=8,
)

In [43]:
for batch in tf_test_dataset:
  print(batch[0]['input_ids'].shape)

(8, 34)
(8, 33)
(8, 33)
(8, 32)
(8, 34)
(8, 47)
(8, 53)
(8, 36)
(8, 52)
(8, 42)
(8, 32)
(8, 42)
(8, 32)
(8, 34)
(8, 41)
(8, 51)
(8, 33)
(8, 42)
(8, 55)
(8, 33)
(8, 37)
(8, 50)
(8, 43)
(8, 40)
(8, 37)
(8, 55)
(8, 45)
(8, 33)
(8, 34)
(8, 30)
(8, 36)
(8, 64)
(8, 33)
(8, 49)
(8, 44)
(8, 31)
(8, 44)
(8, 23)
(8, 49)
(8, 50)
(8, 50)
(8, 28)
(8, 36)
(8, 29)
(8, 40)
(8, 38)
(3, 23)


In [44]:
checkpoint

'xlm-roberta-base'

In [45]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

Some layers of TFXLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [116]:
model.config

XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [46]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay



batch_size = 8
num_epochs = 5
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [47]:
import tensorflow as tf


loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

In [48]:
model.fit(tf_train_dataset, validation_data=tf_test_dataset, epochs=num_epochs)

Epoch 1/5
243/243 [==============================] - 70s 206ms/step - loss: 0.7898 - accuracy: 0.7276 - val_loss: 0.7561 - val_accuracy: 0.7278
Epoch 2/5
243/243 [==============================] - 47s 194ms/step - loss: 0.7743 - accuracy: 0.7281 - val_loss: 0.7684 - val_accuracy: 0.7278
Epoch 3/5
243/243 [==============================] - 47s 194ms/step - loss: 0.7734 - accuracy: 0.7286 - val_loss: 0.7556 - val_accuracy: 0.7278
Epoch 4/5
243/243 [==============================] - 48s 195ms/step - loss: 0.7697 - accuracy: 0.7286 - val_loss: 0.7565 - val_accuracy: 0.7278
Epoch 5/5
243/243 [==============================] - 47s 195ms/step - loss: 0.7681 - accuracy: 0.7286 - val_loss: 0.7561 - val_accuracy: 0.7278


In [99]:
import numpy as np

def predict_emotion(example):
  model_input = tokenizer(example,return_tensors = "tf")
  output = model(**model_input)
  pred = tf.nn.softmax(
   output["logits"], axis=None, name=None
)
  class_preds = np.argmax(pred)
  return class_labels[class_preds]


In [100]:
predict_emotion("এটা ভাল না")

'Negative'

In [101]:
predict_emotion("আজ কিছু করতে ভালো লাগছে না")

'Negative'

In [104]:
predict_emotion("আফতাব আপনি ভালো আছেন?")

'Negative'

In [114]:
train_cat_labels_count

negative    1415
positive     344
neutral      183
Name: class_label, dtype: int64

In [115]:
preds = model.predict(tf_test_dataset)

TFSequenceClassifierOutput(loss=None, logits=array([[-0.97436863, -0.355422  ,  1.0541221 ],
       [-0.9743753 , -0.3553544 ,  1.0539474 ],
       [-0.9743502 , -0.35540187,  1.0540698 ],
       ...,
       [-0.97435355, -0.35542533,  1.0541747 ],
       [-0.9743839 , -0.3553031 ,  1.0540793 ],
       [-0.97437406, -0.3553982 ,  1.0540817 ]], dtype=float32), hidden_states=None, attentions=None)